In [1]:
import datetime as dt

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [3]:
df_elections_dates = pd.read_csv('legislative_elections_dates.csv')

In [4]:
anom_sd1 = pd.read_excel('Anom_SD1.xlsx')

In [5]:
anom_sd1.head()

,Pkey,128,1509,2627,315,2108,527,1132,1209,4823,...,Unnamed: 2308,Unnamed: 2309,Unnamed: 2310,Unnamed: 2311,Unnamed: 2312,Unnamed: 2313,Unnamed: 2314,Unnamed: 2315,Unnamed: 2316,Unnamed: 2317
0,DISTRICT,Adilabad,Agra,Ahmadnagar,Aizawl,Ajmer,Akola,Alappuzha,Aligarh,Alirajpur,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DT_CEN_CD,1,15,26,3,21,5,11,12,48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ST_CEN_CD,28,9,27,15,8,27,32,9,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ST_NM,Andhra Pradesh,Uttar Pradesh,Maharashtra,Mizoram,Rajasthan,Maharashtra,Kerala,Uttar Pradesh,Madhya Pradesh,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Label,NaN,Large City,NaN,NaN,Large City,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
cols = anom_sd1.columns.tolist()

In [7]:
raw = anom_sd1[cols[:cols.index( u'MA(TS-MA)')]]

In [8]:
raw.columns = raw[0:1].values.tolist()

In [9]:
raw = raw[1:]

In [10]:
raw_cols = raw.columns.tolist()

In [11]:
std_all_years = list()

In [12]:
states_list = raw[2:3].values.tolist()[0]

In [13]:
states_not_processed = list()

In [14]:
time_line = raw['DISTRICT'][6:-1].values.tolist()

In [15]:
time_line_years = [time_val.year for time_val in time_line]

In [16]:
time_line_months = [time_val.month for time_val in time_line]

In [17]:
month_name = ['0','jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']

In [18]:
for i in range(len(df_elections_dates)):
    state = df_elections_dates[i:i+1].values[0][0]
    month = df_elections_dates[i:i+1].values[0][1]
    year  = df_elections_dates[i:i+1].values[0][2]

    if not(month in time_line_months and year in time_line_years and state in states_list):
        states_not_processed.append([state,month,year])
        continue

    ## for state
    f_name = str(state)+'_' + str(month) +'_'+str(year)

    title = 'elections: '+ f_name

    file_name = f_name +'.png'

    dist_list = [raw_cols[i] for i,state_ in enumerate(raw[2:3].values.tolist()[0]) if state_==state]

    ## this is night light values according to state
    ntl_values = raw[dist_list][6:-1].sum(axis=1).values.tolist()

    np.shape(ntl_values)

    np.shape(time_line)

    A = range(len(time_line))
    B = ntl_values

    plt.plot(A,B)
    plt.plot(A,B,'ko')
    plt.title(title)
    plt.ylabel('raw_values')
    plt.grid()

    month_dates = list()
    month_vals = list()
    for i,time_value in enumerate(time_line):
        if time_value.month == month:
            ## putting the values in list
            month_vals.append(B[i])

            ## highlighting all that months
            data_label = month_name[time_value.month]+str(time_value.year)[2:]
            month_dates.append(data_label)
            plt.plot(A[i],B[i],'ko',markerfacecolor='None',markeredgecolor='black',markersize=15)
            plt.annotate(data_label,xy=(A[i]+1,B[i]))

            ## highlighting that year
            if time_value.year == year:
                year_val = B[i]
                plt.plot(A[i],B[i],'ks',markerfacecolor='None',markeredgecolor='black',markersize=15)

    month_std = np.std(month_vals,ddof=1)
    month_mean = np.mean(month_vals)

    std_year = (year_val - month_mean)/month_std

    plt.annotate('std: '+str(std_year) ,xy=(50,min(B)))

    std_all_years.append([state,std_year])

    plt.savefig('elections/'+file_name)

    plt.close()

In [24]:
df_stdCounts_allStates = pd.DataFrame(std_all_years)

In [26]:
df_stdCounts_allStates.columns = ['state','stdCount']

In [28]:
df_states_notProcessed = pd.DataFrame(states_not_processed)

In [30]:
df_states_notProcessed.columns = ['States','Month','Year']

In [32]:
df_sotdCounts_allStates.to_csv('elections/stdCounts_states')

In [33]:
df_states_notProcessed.to_csv('elections/notProcessedStates')